In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
sys.path.insert(0, '../gantools/')

import numpy as np
import tensorflow as tf
import functools
from matplotlib import pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from gantools import utils, plot, data, evaluation, blocks
from gantools.gansystem import GANsystem, UpcaleGANsystem
from gantools.model import WGAN, UpscalePatchWGAN, CosmoWGAN



In [ ]:
np.__version__

In [ ]:
# Setting the forward and backward transform

forward = functools.partial(data.fmap.log_norm_forward, c=8000.0, scale=6.0)
backward = functools.partial(data.fmap.log_norm_backward, c=8000.0, scale=6.0)

backward_uniscale = functools.partial(data.fmap.stat_backward, c=20000.0, shift=3)
forward_uniscale = functools.partial(data.fmap.stat_forward, c=20000.0, shift=3)

# number of sample generated
num_samples = 30

pathmodel = '../trained_models/3DCosmo'
pathsample = '../samples/3DCosmo'

In [ ]:
def stat_forward_0(x, c=2e4):
    if not type(x).__module__ == np.__name__:
        x = np.array([x])
    res = np.zeros(shape=x.shape, dtype=np.float32)
    mask = x>c
    maski = mask==False
    res[maski] = np.log(x[maski]+1)
    res[mask] = np.log(c+1) + (x[mask]/c-1)
    res = 3*res / np.log(c+1)
    return res

def stat_backward_0(x, c=2e4):
    if not type(x).__module__ == np.__name__:
        x = np.array([x])
    res = np.zeros(shape=x.shape, dtype=np.float32)
    mc = np.log(c+1)
    x = x*mc/3
    mask = x>mc
    maski = mask==False
    res[maski] = np.exp(x[maski])-1
    res[mask] = c*(x[mask] - np.log(c+1) + 1)
    return np.round(res)

def stat_forward(x, c=2e4, shift=3):
    return stat_forward_0(x+shift, c=c) - stat_forward_0(shift, c=c)


def stat_backward(x, c=2e4, shift=3):
    clip_max = c*100
    return stat_backward_0(x+stat_forward_0(shift, c=c), c=c) - shift

backward_uniscale = stat_backward
forward_uniscale = stat_forward

In [ ]:
dataset256 = data.load.load_dataset(resolution=256,Mpch=350,shuffle=False,forward_map=None,spix=256,patch=False,is_3d=True,augmentation=False)
cubes = dataset256.get_all_data()

In [ ]:
np.sum(np.abs(backward(forward(cubes)) -cubes))

In [ ]:
np.sum(np.abs(backward_uniscale(forward_uniscale(cubes)) -cubes))

In [ ]:
def plot_cubes_paper(cubes, slice_num=None, nx=2, ny=2, bkd=backward, **kwargs):
    def map_to_plot(img):
        return data.fmap.stat_forward(bkd(img))
    if len(cubes)<nx*ny:
        raise ValueError("Not enough samples.")
    if slice_num is None:
        slice_num = cubes.shape[1]//2+4
    fig, ax = plt.subplots(nx, ny, sharey=True,figsize=(11/2*nx,10.5/2*ny))
    sn = 0
    for i in range(nx):
        for j in range(ny):
            if nx==1 and ny==1:
                tax = ax
            elif nx==1:
                tax = ax[j]
            elif ny==1:
                tax = ax[i]
            else:
                tax = ax[i,j]
            tax.imshow(map_to_plot(cubes[sn,slice_num]), interpolation='none', **kwargs)
            tax.axis('off')
            sn += 1
    plt.tight_layout()

    return fig

# Generate samples - 32->64->256

## 0->32

In [ ]:
name = '0_to_32'
checkpoint = 27000

pathmodel32 = os.path.join(pathmodel, name)
params_32 = utils.load_params(pathmodel32)
# params_32 = old2new_params(params_32, CosmoWGAN)
# params_32['net']['shape'] = [*params_32['net']['shape'],1]

# params_32['net']['cosmology']['backward_map'] = backward
# params_32['net']['cosmology']['forward_map'] = forward
# params_32['summary_dir'] = os.path.join(pathmodel, name +'summary/')

# for ind, item in enumerate(params.items()):
#     print(ind, " : ", item[0], " - ", item[1])
# print(resume)


In [ ]:
wgan_32 = GANsystem(CosmoWGAN,params_32, name='WGAN')
gen_samples_32 = wgan_32.generate(N=num_samples, checkpoint=checkpoint)
gen_samples_32 = np.squeeze(gen_samples_32)

In [ ]:
os.makedirs(os.path.join(pathsample,'0_32/'), exist_ok=True)
utils.save_hdf5(gen_samples_32, os.path.join(pathsample,'0_32/32_samples_ckpt_latest.h5'))

## 32->64

Load fake 32

In [ ]:
gen_samples_32 = utils.load_hdf5(os.path.join(pathsample,'0_32/32_samples_ckpt_latest.h5'))[:num_samples]
print("gen_samples_32 shape=", gen_samples_32.shape)
gen_samples_32 = np.expand_dims(gen_samples_32, axis=4)
print('downsampled shape=', gen_samples_32.shape)

In [ ]:
name = '32_to_64'
class CosmoUpscalePatchWGAN(UpscalePatchWGAN, CosmoWGAN):
    pass

pathmodel64 = os.path.join(pathmodel, name)
params_64 = utils.load_params(pathmodel64)
params_64['net']['generator']['use_old_gen'] = True

In [ ]:
wgan_64 = UpcaleGANsystem(CosmoUpscalePatchWGAN, params_64)
gen_samples_64 = wgan_64.upscale_image(small=gen_samples_32, checkpoint=None)

In [ ]:
os.makedirs(os.path.join(pathsample,'32_64/'), exist_ok=True)
utils.save_hdf5(gen_samples_64, os.path.join(pathsample,'32_64/64_samples_ckpt_latest.h5'))

In [ ]:
# cmin = 0
# cmax = 1.6
# clim = (cmin, cmax)
# plot_cubes_paper(gen_samples_64, cmap=plt.cm.plasma, clim=clim, nx=4, ny=4);


In [ ]:

# import pickle
# path_param = os.path.join(pathmodel64, 'params.pkl')
# with open(path_param, 'wb') as f:
#     pickle.dump(params_64, f)

## 64->256

In [ ]:
gen_samples_64 = utils.load_hdf5(os.path.join(pathsample,'32_64/64_samples_ckpt_latest.h5'))

print('gen_samples_64 shape=', gen_samples_64.shape)
gen_samples_64 = np.expand_dims(gen_samples_64, axis=4)[:num_samples]
print('downsampled shape=', gen_samples_64.shape)

In [ ]:
name = '64_to_256'
pathmodel256 = os.path.join(pathmodel, name)

params_256 = utils.load_params(pathmodel256)
# params_256['net']['generator']['use_old_gen'] = True
# params_256 = old2new_params(params_256_old, CosmoUpscalePatchWGAN, checky=True)
# params_256['net']['cosmology']['backward_map'] = backward
# params_256['net']['cosmology']['forward_map'] = forward
# params_256['save_dir'] = pathmodel256

In [ ]:
wgan_256 = UpcaleGANsystem(CosmoUpscalePatchWGAN, params_256)
gen_samples_256 = wgan_256.upscale_image(small=gen_samples_64, checkpoint=None)

In [ ]:
# cmin = 0
# cmax = 3
# clim = (cmin, cmax)
# plot_cubes_paper(gen_samples_256, cmap=plt.cm.plasma, clim=clim, nx=2, ny=2);

In [ ]:
os.makedirs(os.path.join(pathsample,'64_256/'), exist_ok=True)
utils.save_hdf5(gen_samples_256, os.path.join(pathsample,'64_256/256_samples_ckpt_latest.h5'))

In [ ]:
# import pickle
# path_param = os.path.join(pathmodel256, 'params.pkl')
# with open(path_param, 'wb') as f:
#     pickle.dump(params_256, f)

# Uniscale model
This is not working. There is a special branch for this model.

In [ ]:
name = 'uni_scale'
pathmodel_uniscale = os.path.join(pathmodel, name)
params_uniscale = utils.load_params(pathmodel_uniscale)
class CosmoUpscalePatchWGAN(UpscalePatchWGAN, CosmoWGAN):
    pass
# params_uniscale = old2new_params(params_uniscale_old, CosmoUpscalePatchWGAN, checky=True)

# params_uniscale['net']['cosmology']['backward_map'] = backward_uniscale
# params_uniscale['net']['cosmology']['forward_map'] = forward_uniscale

# params_uniscale['save_dir'] = pathmodel_uniscale
# params_uniscale['net']['generator']['residual'] = False
# params_uniscale['net']['generator']['use_conv_over_deconv'] = False


In [ ]:
wgan_uniscale = UpcaleGANsystem(CosmoUpscalePatchWGAN, params_uniscale)
gen_samples_uniscale = wgan_uniscale.upscale_image(N=num_samples, resolution=256)

In [ ]:
savedat = np.load('testgen2.npz')
res = savedat['res'][0]
res.shape
res_raw = savedat['res'][1]



In [ ]:
res2 = wgan_uniscale.generate(z=savedat['z'], borders=savedat['borders'])
plt.imshow(res2[0,:,:,10,0])
plt.figure()
plt.imshow(res[0,:,:,10,0])
np.testing.assert_almost_equal(res,res2, decimal=5)


# res_raw[np.abs(res_raw-backward_uniscale(res2))>0]

In [ ]:
# params_uniscale['net']['cosmology']['backward_map'] = backward_uniscale
# params_uniscale['net']['cosmology']['forward_map'] = forward_uniscale
# import pickle
# path_param = os.path.join(pathmodel_uniscale, 'params.pkl')
# with open(path_param, 'wb') as f:
#     pickle.dump(params_uniscale, f)

In [ ]:
os.makedirs(os.path.join(pathsample,'uniscale/'), exist_ok=True)
utils.save_hdf5(gen_samples_uniscale, os.path.join(pathsample,'uniscale/256_samples_ckpt_latest.h5'))

In [ ]:
cmin = 0
cmax = 3
clim = (cmin, cmax)
plot_cubes_paper(gen_samples_uniscale, cmap=plt.cm.plasma, clim=clim, nx=2, ny=2, bkd=backward_uniscale);

In [ ]:

# def upscale_image(obj, small=None, num_samples=None, resolution=None, checkpoint=None, is_3d=False):
#     """Upscale image using the lappachsimple model, or upscale_WGAN_pixel_CNN model.

#     For model upscale_WGAN_pixel_CNN, pass num_samples to generate and resolution of the final bigger histogram.
#     for model lappachsimple         , pass small.

#     3D only works for upscale_WGAN_pixel_CNN.

#     This function can be accelerated if the model is created only once.
#     """
#     # Number of sample to produce
#     if small is None:
#         if num_samples is None:
#             raise ValueError("Both small and num_samples cannot be None")
#         else:
#             N = num_samples
#     else:
#         N = small.shape[0]

#     # Output dimension of the generator
#     soutx, souty = obj.params['net']['shape'][:2]
#     if is_3d:
#         soutz = obj.params['net']['shape'][2]

#     if small is not None:
#         # Dimension of the low res image
#         lx, ly = small.shape[1:3]
#         if is_3d:
#             lz = small.shape[3]

#         # Input dimension of the generator
#         sinx = soutx // obj.params['generator']['downsampling']
#         siny = souty // obj.params['generator']['downsampling']
#         if is_3d:
#             sinz = soutz // obj.params['generator']['downsampling']

#         # Number of part to be generated
#         nx = lx // sinx
#         ny = ly // siny
#         if is_3d:
#             nz = lz // sinz

#     else:
#         sinx = siny = sinz = None
#         if resolution is None:
#             raise ValueError("Both small and resolution cannot be None")
#         else:
#             nx = resolution // soutx
#             ny = resolution // souty
#             nz = resolution // soutz


#     # Final output image
#     with tf.Session() as sess:
#         obj.load(sess=sess)

#         if is_3d:
#             output_image = generate_3d_output(sess, obj, N, nx, ny, nz, soutx, souty, soutz, small, sinx, siny, sinz)
#         else:
#             output_image = generate_2d_output(sess, obj, N, nx, ny, soutx, souty, small, sinx, siny)
        
#     return np.squeeze(output_image)


# def generate_3d_output(sess, obj, N, nx, ny, nz, soutx, souty, soutz, small, sinx, siny, sinz):
#     output_image = np.zeros(
#             shape=[N, soutz * nz, souty * ny, soutx * nx, 1], dtype=np.float32)
#     output_image[:] = np.nan

#     print('Total number of patches = {}*{}*{} = {}'.format(nx, ny, nz, nx*ny*nz) )

#     for k in range(nz): # height
#         for j in range(ny): # rows
#             for i in range(nx): # columns

#                 # 1) Generate the border
#                 border = np.zeros([N, soutz, souty, soutx, 7])

#                 if j: # one row above, same height
#                     border[:, :, :, :, 0:1] = output_image[:, 
#                                                             k * soutz:(k + 1) * soutz,
#                                                             (j - 1) * souty:j * souty, 
#                                                             i * soutx:(i + 1) * soutx, 
#                                                         :]
#                 if i: # one column left, same height
#                     border[:, :, :, :, 1:2] = output_image[:,
#                                                             k * soutz:(k + 1) * soutz,
#                                                             j * souty:(j + 1) * souty, 
#                                                             (i - 1) * soutx:i * soutx, 
#                                                         :]
#                 if i and j: # one row above, one column left, same height
#                     border[:, :, :, :, 2:3] = output_image[:,
#                                                             k * soutz:(k + 1) * soutz,
#                                                             (j - 1) * souty:j * souty, 
#                                                             (i - 1) * soutx:i * soutx, 
#                                                         :]
#                 if k: # same row, same column, one height above
#                     border[:, :, :, :, 3:4] = output_image[:,
#                                                             (k - 1) * soutz:k * soutz,
#                                                             j * souty:(j + 1) * souty, 
#                                                             i * soutx:(i + 1) * soutx, 
#                                                         :]
#                 if k and j: # one row above, same column, one height above
#                     border[:, :, :, :, 4:5] = output_image[:,
#                                                             (k - 1) * soutz:k * soutz,
#                                                             (j - 1) * souty:j * souty, 
#                                                             i * soutx:(i + 1) * soutx, 
#                                                         :]
#                 if k and i: # same row, one column left, one height above
#                     border[:, :, :, :, 5:6] = output_image[:,
#                                                             (k - 1) * soutz:k * soutz,
#                                                             j * souty:(j + 1) * souty, 
#                                                             (i - 1) * soutx:i * soutx, 
#                                                         :]
#                 if k and i and j: # one row above, one column left, one height above
#                     border[:, :, :, :, 6:7] = output_image[:,
#                                                             (k - 1) * soutz:k * soutz,
#                                                             (j - 1) * souty:j * souty, 
#                                                             (i - 1) * soutx:i * soutx, 
#                                                         :]


#                 # 2) Prepare low resolution
#                 if small is not None:
#                     downsampled = small[:, k * sinx:(k + 1) * sinz, 
#                                            j * siny:(j + 1) * siny,
#                                            i * siny:(i + 1) * sinx,
#                                            :]
#                 else:
#                     downsampled = None

#                 # 3) Generate the image
#                 print('Current patch: column={}, row={}, height={}'.format(i+1, j+1, k+1))
#                 gen_sample = obj.generate(
#                     N=N, borders=border, X_down=downsampled, sess=sess)

#                 output_image[:, 
#                                 k * soutz:(k + 1) * soutz,
#                                 j * souty:(j + 1) * souty, 
#                                 i * soutx:(i + 1) * soutx, 
#                             :] = gen_sample

#     return output_image


In [ ]:
# gen_samples_uniscale = upscale_image(wgan_uniscale, num_samples=num_samples, resolution=256, checkpoint='38000', is_3d=True)


# Make the figures for the paper

In [ ]:
pathfig = 'figures/'
os.makedirs(pathfig, exist_ok=True)

#### Load all samples

In [ ]:
gen_samples_32 = utils.load_hdf5(os.path.join(pathsample,'0_32/32_samples_ckpt_latest.h5'))
gen_samples_64 = utils.load_hdf5(os.path.join(pathsample,'32_64/64_samples_ckpt_latest.h5'))
gen_samples_256 = utils.load_hdf5(os.path.join(pathsample,'64_256/256_samples_ckpt_latest.h5'))
gen_samples_uniscale = utils.load_hdf5(os.path.join(pathsample,'uniscale/256_samples_ckpt_latest.h5'))


#### Backward transform

In [ ]:
gen_samples_32_raw = backward(gen_samples_32)
gen_samples_64_raw = backward(gen_samples_64)
gen_samples_256_raw = backward(gen_samples_256)


In [ ]:
# specialpath = '/store/nati/archive/cosmosamples/uniscale/256_samples_ckpt_latest.h5'
# gen_samples_uniscale = utils.load_hdf5(specialpath)

In [ ]:
gen_samples_uniscale_raw = backward_uniscale(gen_samples_uniscale)


#### Load real data

In [ ]:
dataset256 = data.load.load_dataset(resolution=256,Mpch=350,shuffle=False,forward_map=forward,spix=256,patch=False,is_3d=True,augmentation=False)
real_samples_256 = dataset256.get_all_data()
del dataset256


In [ ]:
real_samples_256_raw = backward(real_samples_256)

In [ ]:
dataset32 = data.load.load_dataset(resolution=256,Mpch=350,shuffle=False,forward_map=forward,spix=32,patch=False,is_3d=True,augmentation=False, scaling=8)
real_samples_32 = dataset32.get_all_data()
del dataset32

In [ ]:
real_samples_32_raw = backward(real_samples_32)

In [ ]:
dataset64 = data.load.load_dataset(resolution=256,Mpch=350,shuffle=False,forward_map=forward,spix=64,patch=False,is_3d=True,augmentation=False, scaling=4)
real_samples_64 = dataset64.get_all_data()
del dataset64

In [ ]:
real_samples_64_raw = backward(real_samples_64)

In [ ]:
# cmin = np.min(cubes)
# cmax = np.max(cubes)/1.5
cmin = 0
cmax = 3
clim = (cmin, cmax)
fig = plot_cubes_paper(real_samples_256, cmap=plt.cm.plasma, clim=clim);
fig.suptitle('Real $256^3$', y=1.02, fontsize=24 )
plt.savefig(pathfig+"real256.pdf", bbox_inches='tight', format='pdf')

fig2 = plot_cubes_paper(gen_samples_256, cmap=plt.cm.plasma, clim=clim);
fig2.suptitle('Fake $256^3$', y=1.02, fontsize=24 )
plt.savefig(pathfig+"fake256.pdf", bbox_inches='tight', format='pdf')



In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(real_samples_256_raw, gen_samples_256_raw)
plt.savefig(pathfig+"256full_hist.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(real_samples_256_raw, gen_samples_256_raw)
plt.savefig(pathfig+"256full_peak.pdf", bbox_inches='tight', format='pdf')

In [ ]:
evaluation.compute_and_plot_psd(real_samples_256_raw, gen_samples_256_raw)
plt.savefig(pathfig+"256full_psd.pdf", bbox_inches='tight', format='pdf')

# Scale by scale analysis

## 32 cubes

In [ ]:
# cmin = np.min(cubes_32)
# # cmin = 0
# cmax = np.max(cubes_32)
cmin = 0.1
cmax = 1.3
clim = (cmin, cmax)
fig = plot_cubes_paper(real_samples_32, cmap=plt.cm.plasma, clim=clim, nx=4, ny=4);
fig.suptitle('Real $32^3$', y=1.03, fontsize=48 )
plt.savefig(pathfig+"real32.pdf", bbox_inches='tight', format='pdf')

fig2 = plot_cubes_paper(gen_samples_32, cmap=plt.cm.plasma, clim=clim, nx=4, ny=4);
fig2.suptitle('Fake $32^3$', y=1.03, fontsize=48 )
plt.savefig(pathfig+"fake32.pdf", bbox_inches='tight', format='pdf')



In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(real_samples_32_raw, gen_samples_32_raw)
plt.savefig(pathfig+"32_hist.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(real_samples_32_raw, gen_samples_32_raw)
plt.savefig(pathfig+"32_peak.pdf", bbox_inches='tight', format='pdf')

In [ ]:
evaluation.compute_and_plot_psd(real_samples_32_raw, gen_samples_32_raw)
plt.savefig(pathfig+"32_psd.pdf", bbox_inches='tight', format='pdf')

## 64 Cubes

In [ ]:
name = '32_to_64'
class CosmoUpscalePatchWGAN(UpscalePatchWGAN, CosmoWGAN):
    pass

pathmodel64 = os.path.join(pathmodel, name)
params_64 = utils.load_params(pathmodel64)
wgan_64 = UpcaleGANsystem(CosmoUpscalePatchWGAN, params_64)
gen_samples_64_single = wgan_64.upscale_image(small=np.reshape(real_samples_32, [*real_samples_32.shape,1]))

In [ ]:
gen_samples_64_single_raw = backward(gen_samples_64_single)

In [ ]:
# cmin = np.min(cubes_64)
# cmax = np.max(cubes_64)
cmin = 0
cmax = 1.6
clim = (cmin, cmax)
fig = plot_cubes_paper(real_samples_64, cmap=plt.cm.plasma, clim=clim);
fig.suptitle('Real $64^3$', y=1.04, fontsize=36 )
plt.savefig(pathfig+"up_real64.pdf", bbox_inches='tight', format='pdf')

fig2 = plot_cubes_paper(gen_samples_64_single, cmap=plt.cm.plasma, clim=clim);
fig2.suptitle('Fake $64^3$', y=1.04, fontsize=36 )
plt.savefig(pathfig+"up_fake64.pdf", bbox_inches='tight', format='pdf')

fig = plot_cubes_paper(real_samples_32, cmap=plt.cm.plasma, clim=clim);
fig.suptitle('Real downsampled $32^3$', y=1.04, fontsize=36 )
plt.savefig(pathfig+"up_down32.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(real_samples_64_raw, gen_samples_64_single_raw)
plt.savefig(pathfig+"up_64_hist.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(real_samples_64_raw, gen_samples_64_single_raw)
plt.savefig(pathfig+"up_64_peak.pdf", bbox_inches='tight', format='pdf')

In [ ]:
evaluation.compute_and_plot_psd(real_samples_64_raw, gen_samples_64_single_raw)
plt.savefig(pathfig+"up_64_psd.pdf", bbox_inches='tight', format='pdf')

## 256 Cubes

In [ ]:
name = '64_to_256'
pathmodel256 = os.path.join(pathmodel, name)

params_256 = utils.load_params(pathmodel256)

wgan_256 = UpcaleGANsystem(CosmoUpscalePatchWGAN, params_256)
gen_samples_single_256 = wgan_256.upscale_image(small=np.reshape(real_samples_64, [*real_samples_64.shape, 1]))

In [ ]:
# cmin = np.min(cubes_256)
# cmax = np.max(cubes_256)/2.5
cmin = 0
cmax = 3
clim = (cmin, cmax)
fig = plot_cubes_paper(real_samples_256, cmap=plt.cm.plasma, clim=clim, ny=1);
fig.suptitle('Real $256^3$', y=1.03, fontsize=12 )
plt.savefig(pathfig+"up_real256.pdf", bbox_inches='tight', format='pdf')

fig2 = plot_cubes_paper(gen_samples_single_256, cmap=plt.cm.plasma, clim=clim, ny=1);
fig2.suptitle('Fake $256^3$', y=1.03, fontsize=12 )
plt.savefig(pathfig+"up_fake256.pdf", bbox_inches='tight', format='pdf')

fig = plot_cubes_paper(real_samples_64, cmap=plt.cm.plasma, clim=clim, ny=1);
fig.suptitle('Real downsampled $64^3$', y=1.03, fontsize=12 )
plt.savefig(pathfig+"up_down64.pdf", bbox_inches='tight', format='pdf')

In [ ]:
gen_samples_single_256_raw = backward(gen_samples_single_256)

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(real_samples_256_raw, gen_samples_single_256_raw)
plt.savefig(pathfig+"up_256_hist.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(real_samples_256_raw, gen_samples_single_256_raw)
plt.savefig(pathfig+"up_256_peak.pdf", bbox_inches='tight', format='pdf')

In [ ]:
evaluation.compute_and_plot_psd(real_samples_256_raw, gen_samples_single_256_raw)
plt.savefig(pathfig+"up_256_psd.pdf", bbox_inches='tight', format='pdf')

## Uniscale

In [ ]:
cmin = 0
cmax = 3
clim = (cmin, cmax)
fig = plot_cubes_paper(real_samples_256, cmap=plt.cm.plasma, clim=clim);
fig.suptitle('Real $256^3$', y=1.02, fontsize=24 )
plt.savefig(pathfig+"uniscalereal256.pdf", bbox_inches='tight', format='pdf')

fig2 = plot_cubes_paper(gen_samples_uniscale, cmap=plt.cm.plasma, clim=clim, bkd=backward_uniscale);
fig2.suptitle('Fake $256^3 - uniscale$', y=1.02, fontsize=24 )
plt.savefig(pathfig+"uniscalefake256.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(real_samples_256_raw, gen_samples_uniscale_raw)
plt.savefig(pathfig+"256uniscale_hist.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(real_samples_256_raw, gen_samples_uniscale_raw)
plt.savefig(pathfig+"256uniscale_peak.pdf", bbox_inches='tight', format='pdf')

In [ ]:
evaluation.compute_and_plot_psd(real_samples_256_raw, gen_samples_uniscale_raw)
plt.savefig(pathfig+"256uniscale_psd.pdf", bbox_inches='tight', format='pdf')

# Gifs

## Real 256 cubes

In [ ]:
# fps=16
# clim = (0,3)
# plot.animate_cubes(real_samples_256, output_name=pathfig+"real256.gif", clim=clim, fps=fps, cmap=plt.cm.plasma)
# animation = plot.cubes_to_animation(real_samples_256, cmap=plt.cm.plasma, clim=clim)
# animation.ipython_display(fps=16, loop=True, autoplay=True)


In [ ]:
# plot.save_animation(real_samples_256[0], gen_samples_256[0], figsize=(10, 6), fps=16, format='mp4', output_file_name=pathfig+'final.mp4', clim=clim)
# plt.style.use('ggplot')